
# 07 — Agents Projects Lab

This notebook contains **three agentic project skeletons** you can extend:

1. 🛡️ Security Auditor Agent (log/config scanner style)
2. 💻 Code Reviewer Agent (pull-request / snippet review)
3. 📚 Research Copilot Agent (multi-document research helper)

Each section is runnable and uses LangChain + simple agent-style functions.
You can later upgrade them into full LangGraph / MCP workflows.


## 0. Common Setup

In [ ]:

!pip install langchain langchain-openai chromadb langchain-community --quiet

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma

llm = ChatOpenAI(model="gpt-4o-mini")
emb = OpenAIEmbeddings()


## 1. 🛡️ Security Auditor Agent (Skeleton)


### Goal

A simple agent that:

- Loads **security-related config/log snippets**
- Uses RAG-style retrieval
- Applies a **SecurityAuditorAgent** that looks for risky patterns


In [ ]:

import os
os.makedirs("agents_lab_data", exist_ok=True)

with open("agents_lab_data/security_snippets.txt", "w") as f:
    f.write("DEBUG: API key exposed in logs for service X.\n")
    f.write("WARNING: Using HTTP instead of HTTPS on login endpoint.\n")
    f.write("INFO: Password policy requires at least 8 characters but no special chars.\n")

loader_sec = TextLoader("agents_lab_data/security_snippets.txt")
docs_sec = loader_sec.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks_sec = splitter.split_documents(docs_sec)

db_sec = Chroma.from_documents(chunks_sec, emb)
retriever_sec = db_sec.as_retriever(search_kwargs={"k": 3})


In [ ]:

class SecurityAuditorAgent:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm

    def run(self, question: str) -> str:
        docs = self.retriever.get_relevant_documents(question)
        ctx = "\n".join(d.page_content for d in docs)
        prompt = f"""You are a security auditor.
Use ONLY the context below (logs/configs).

Context:
{ctx}

Question: {question}

List potential security risks in bullet points. If nothing obvious, say so.
"""
        return self.llm.invoke(prompt).content

sec_agent = SecurityAuditorAgent(retriever_sec, llm)
print(sec_agent.run("Do you see any security issues here?"))


## 2. 💻 Code Reviewer Agent (Skeleton)


### Goal

A simple agent that:

- Takes a code snippet as input
- Reviews it for readability, style, and potential bugs
- Returns structured feedback


In [ ]:

class CodeReviewerAgent:
    def __init__(self, llm):
        self.llm = llm

    def run(self, code: str, language: str = "python") -> str:
        prompt = f"""You are a senior {language} engineer.
Review the code snippet and provide:
- Summary of behavior
- Potential bugs/edge cases
- Suggestions for readability/style

Code:
```{language}
{code}
```

Answer in bullet points.
"""
        return self.llm.invoke(prompt).content

code_snippet = """
def add_items(items, to_add=[]):
    for i in items:
        to_add.append(i)
    return to_add
"""

code_reviewer = CodeReviewerAgent(llm)
print(code_reviewer.run(code_snippet, language="python"))


## 3. 📚 Research Copilot Agent (Skeleton)


### Goal

A simple research copilot that:

- Loads a small set of topic snippets
- Retrieves context
- Produces a short structured summary


In [ ]:

with open("agents_lab_data/research_notes.txt", "w") as f:
    f.write("RAG combines retrieval of external documents with language model generation.\n")
    f.write("Agents can plan, decide which tools to call, and route queries.\n")
    f.write("LangGraph enables graph-based workflows for multi-agent systems.\n")

loader_res = TextLoader("agents_lab_data/research_notes.txt")
docs_res = loader_res.load()

splitter_res = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks_res = splitter_res.split_documents(docs_res)

db_res = Chroma.from_documents(chunks_res, emb)
retriever_res = db_res.as_retriever(search_kwargs={"k": 3})


In [ ]:

class ResearchCopilotAgent:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm

    def run(self, question: str) -> str:
        docs = self.retriever.get_relevant_documents(question)
        ctx = "\n".join(d.page_content for d in docs)
        prompt = f"""You are a research copilot.
Use ONLY the context below.

Context:
{ctx}

Question: {question}

Provide a 3-5 bullet-point answer.
"""
        return self.llm.invoke(prompt).content

research_agent = ResearchCopilotAgent(retriever_res, llm)
print(research_agent.run("What is the relationship between RAG and agents?"))



## End of Notebook 07 — Agents Projects Lab

You now have three runnable agent skeletons:
- SecurityAuditorAgent
- CodeReviewerAgent
- ResearchCopilotAgent

Extend each into full LangGraph + MCP workflows later.
